In [1]:
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn') #visual style
sns.set(font_scale=2.5)


#ignore warning
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
df=pd.read_csv("titanic_train.csv")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [5]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
sex_age_mean=df.groupby("Sex").Age.mean()

In [7]:
df.loc[(df.Sex=="male")&(df["Age"].isnull()),"Age"]=sex_age_mean[1]

In [8]:
df.loc[(df.Sex=="female")&(df["Age"].isnull()),"Age"]=sex_age_mean[0]

In [9]:
df["Age"].isnull().sum()

0

In [10]:
df["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [11]:
df["Embarked"].fillna("S",inplace=True)

In [12]:
df.drop(["Cabin"],axis=1,inplace=True)

In [13]:
df["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [14]:
df["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [15]:
df["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [16]:
df["Sex"]=df["Sex"].map({"male":1,"female":0})
df["Sex"].value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [17]:
df["Embarked"].value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [18]:
df=pd.get_dummies(df,columns=["Embarked"])

In [19]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,0,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,0,0,1
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,0,0,1


In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [21]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [22]:
X = df [['Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S']]

In [23]:
y = df['Survived']

In [24]:
clf = SVC(kernel='linear', C=1)

In [25]:
scores = cross_val_score(clf, X, y, cv=5)
scores

array([0.80446927, 0.81005587, 0.78651685, 0.75280899, 0.78531073])

In [26]:
scores.mean()

0.7878323433649421

In [27]:
scores.std()

0.02002696581431805

In [28]:
from sklearn.model_selection import GridSearchCV

In [29]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [43]:
clf = GridSearchCV(
        SVC(), tuned_parameters,scoring='accuracy',cv=5)

In [44]:
clf.fit(X,y)


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'kernel': ['rbf'], 'gamma': [0.001, 0.0001], 'C': [1, 10, 100, 1000]}, {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_

In [33]:
from sklearn.naive_bayes import BernoulliNB

In [34]:
params = {'alpha': [0.01, 0.1, 0.5, 1.0, 10.0],}

In [35]:
bernoulli_nb_grid = GridSearchCV(BernoulliNB(), param_grid=params, n_jobs=-1, cv=5, verbose=5,scoring='accuracy')

In [36]:
bernoulli_nb_grid.fit(X,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   35.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'alpha': [0.01, 0.1, 0.5, 1.0, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=5)

In [37]:
bernoulli_nb_grid.best_params_

{'alpha': 10.0}

In [38]:
bernoulli_nb_grid.best_score_

0.7789001122334456

In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
m = RandomForestClassifier(n_estimators=200,min_samples_leaf=3,max_features=0.5,n_jobs=-1)

In [41]:
m.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=0.5, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [42]:
m.score(X,y)

0.8967452300785634

In [45]:
from sklearn.externals import joblib
joblib.dump(m,'Titanicmodel.obj')

['Titanicmodel.obj']